## Kaggle Competition - House Prices: Advanced Regression Techniques
# Feature Encoding

The main purpose of this section is to individually review each of the columns in the dataset to see if they can be improved for the purposes of linear regression

The code for this module is in: [src/models/FeatureEncoding.py](../src/models/FeatureEncoding.py)

In [1]:
import sys
import os
sys.path.append( os.path.abspath( os.path.join(os.getcwd(), ".." ))) 
from src.utils import reset_root_dir
reset_root_dir()

from cytoolz import groupby
from datetime import datetime
from pandas import DataFrame, Series
from sortedcontainers import SortedDict
import cytoolz
import itertools
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint
import pydash
import seaborn as sns
import simplejson

from src.utils.Charts import Charts
from src.models import LinearRegressionModel, FeatureEncoding, SquaredFeatureEncoding

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

pp = pprint.PrettyPrinter(depth=6)

# Column Categorization and Cleanup

The baseline linear regression model removes non-numeric fields and fills in NaNs with 0

Looking through the individual columns, there are several types of basic feature mapping

Uncorrilated Variables that can be removed
- id: uncorrilated remove

Linear Numeric fields that left unmodified
- LotFrontage: linear numeric
- LotArea: linear numeric
- OverallQual: linear numeric
- OverallCond: linear numeric
- MasVnrArea: linear numeric
- BsmtFinSF2: linear numeric
- BsmtUnfSF: linear numeric
- TotalBsmtSF: linear numeric
- 1stFlrSF: linear numeric
- 2ndFlrSF: linear numeric
- LowQualFinSF: linear numeric
- GrLivArea: linear numeric
- BsmtFullBath: linear numeric
- BsmtHalfBath: linear numeric
- FullBath: linear numeric
- HalfBath: linear numeric
- Bedroom: linear numeric
- Kitchen: linear numeric
- TotRmsAbvGrd: linear numeric
- WoodDeckSF: linear numeric
- OpenPorchSF: linear numeric
- EnclosedPorch: linear numeric
- 3SsnPorch: linear numeric
- ScreenPorch: linear numeric
- PoolArea: linear numeric
- MiscVal: linear numeric
- Fireplaces: linear numeric
- GarageCars: linear numeric
- GarageArea: linear numeric


Absolute Years could be better converted to Ages Relative to current Year
- YearBuilt: numeric - convert to age
- YearRemodAdd: numeric - convert to age
- GarageYrBlt: numeric - convert to age
- YrSold: numeric - convert to age

Month sold is categorical assuming that houe prices have seasons 
- MoSold: categorical

Categorical fields may need OneHotEncoding - https://scikit-learn.org/stable/modules/preprocessing.html
- MSSubClass: categorical
- MSZoning: categorical
- Street: categorical
- Alley: categorical
- LotShape: categorical
- LandContour: categorical
- Utilities: categorical
- LotConfig: categorical
- Neighborhood: categorical
- BldgType: categorical
- HouseStyle: categorical
- RoofStyle: categorical
- RoofMatl: categorical
- MasVnrType: categorical
- Foundation: categorical
- BsmtExposure: categorical
- Heating: categorical
- Functional: categorical | categorical to numeric
- GarageFinish: categorical | categorical to numeric
- Fence: categorical | categorical to numeric
- SaleType: categorical
- SaleCondition: categorical
- MiscFeature: categorical
- Electrical: categorical
- GarageType: categorical
- PavedDrive: (Yes / Partial / No )

Categorical fields with optional multiple values
- Condition1 + Condition2: multiple categorical
- Exterior1st + Exterior2nd: multiple categorical
- BsmtFinType1 + BsmtFinType2: multiple categorical

LabelEncoder Quality Categories: Ex:Excellent, Gd:Good, TA:Average/Typical, Fa:Fair, Po:Poor, NA
- ExterQual: categorical to numeric
- ExterCond: categorical to numeric
- BsmtQual: categorical to numeric
- BsmtCond: categorical to numeric
- HeatingQC: categorical to numeric
- KitchenQual: categorical to numeric
- FireplaceQu: categorical to numeric
- GarageQual: categorical to numeric
- GarageCond: categorical to numeric
- PoolQC: categorical to numeric

Boolean fields are Y or N which can converted to numeric
- CentralAir: boolean




In [2]:
LinearRegressionModel().data['X_test']

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,882.0,896,0,0,896,0.0,0.0,1,0,2,1,5,0,1961.0,1.0,730.0,140,0,0,0,120,0,0,6,2010
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,1329.0,1329,0,0,1329,0.0,0.0,1,1,3,1,6,0,1958.0,1.0,312.0,393,36,0,0,0,0,12500,6,2010
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,928.0,928,701,0,1629,0.0,0.0,2,1,3,1,6,1,1997.0,2.0,482.0,212,34,0,0,0,0,0,3,2010
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,324.0,926.0,926,678,0,1604,0.0,0.0,2,1,3,1,7,1,1998.0,2.0,470.0,360,36,0,0,0,0,0,6,2010
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,1017.0,1280.0,1280,0,0,1280,0.0,0.0,2,0,2,1,5,0,1992.0,2.0,506.0,0,82,0,0,144,0,0,1,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,21.0,1936,4,7,1970,1970,0.0,0.0,0.0,546.0,546.0,546,546,0,1092,0.0,0.0,1,1,3,1,5,0,0.0,0.0,0.0,0,0,0,0,0,0,0,6,2006
1455,2916,160,21.0,1894,4,5,1970,1970,0.0,252.0,0.0,294.0,546.0,546,546,0,1092,0.0,0.0,1,1,3,1,6,0,1970.0,1.0,286.0,0,24,0,0,0,0,0,4,2006
1456,2917,20,160.0,20000,5,7,1960,1996,0.0,1224.0,0.0,0.0,1224.0,1224,0,0,1224,1.0,0.0,1,0,4,1,7,1,1960.0,2.0,576.0,474,0,0,0,0,0,0,9,2006
1457,2918,85,62.0,10441,5,5,1992,1992,0.0,337.0,0.0,575.0,912.0,970,0,0,970,0.0,1.0,1,0,3,1,6,0,0.0,0.0,0.0,80,32,0,0,0,0,700,7,2006


After feature encoding, we have gone from 37 features to 254 features

In [3]:
FeatureEncoding().data['X_test']

,Id,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,YrSold,YearBuilt_Age,YearRemodAdd_Age,GarageYrBlt_Age,YrSold_Age,CentralAir_Numeric,ExterQual_Numeric,ExterCond_Numeric,BsmtQual_Numeric,BsmtCond_Numeric,HeatingQC_Numeric,KitchenQual_Numeric,FireplaceQu_Numeric,GarageQual_Numeric,GarageCond_Numeric,PoolQC_Numeric,MoSold_1,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,MiscFeature_Gar2,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Condition_Artery,Condition_Feedr,Condition_Norm,Condition_PosA,Condition_PosN,Condition_RRAe,Condition_RRAn,Condition_RRNe,Condition_RRNn,Exterior_AsbShng,Exterior_AsphShn,Exterior_Brk Cmn,Exterior_BrkComm,Exterior_BrkFace,Exterior_CBlock,Exterior_CemntBd,Exterior_CmentBd,Exterior_HdBoard,Exterior_ImStucc,Exterior_MetalSd,Exterior_Other,Exterior_Plywood,Exterior_Stone,Exterior_Stucco,Exterior_VinylSd,Exterior_Wd Sdng,Exterior_Wd Shng,Exterior_WdShing,BsmtFinType_ALQ,BsmtFinType_BLQ,BsmtFinType_GLQ,BsmtFinType_LwQ,BsmtFinType_Rec,BsmtFinType_Unf
0,1461,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,8

# Scoring

We can test each of the feature encodings indivudally as well as compare against a baseline 

Findings:
- (0.1978) X_feature_exclude      - strange corrilation between id and price??? = remove id on therotical grounds
- (0.1946) X_feature_year_ages    - minor reduction in score encoding relative ages = leave absolute year as feature
- (0.1942) none                   - baseline linear regression score
- (0.1855) X_feature_label_encode - significant improvement over baseline score
- (0.1829) X_feature_onehot       - significant improvement over baseline score - better than X_feature_label_encode
- (0.1823) all features           - combining all features doesn't get much better than X_feature_onehot alone

In [4]:
results = FeatureEncoding.feature_scores([ "X_feature_exclude", "X_feature_year_ages", "X_feature_label_encode", "X_feature_onehot" ], depth=1)
for result in reversed(results): print( result )

(0.1978, 'FeatureEncoding', 'LinearRegression', 'X_feature_exclude')
(0.1946, 'FeatureEncoding', 'LinearRegression', 'X_feature_year_ages')
(0.1942, 'FeatureEncoding', 'LinearRegression', '')
(0.1855, 'FeatureEncoding', 'LinearRegression', 'X_feature_label_encode')
(0.1829, 'FeatureEncoding', 'LinearRegression', 'X_feature_onehot')
(0.1823, 'FeatureEncoding', 'LinearRegression', 'X_feature_exclude X_feature_year_ages X_feature_label_encode X_feature_onehot')


### Submit to Kaggle
https://www.kaggle.com/c/house-prices-advanced-regression-techniques/submissions

```
$ kaggle competitions submit -c house-prices-advanced-regression-techniques -f data/submissions/FeatureEncoding.csv -m "FeatureEncoding.py - label + onehot encoding"
```
    
- Your submission scored 0.80406, which is not an improvement of your best score (0.20892). Keep trying!
- Kaggle Rank 3778 / 4375


Whilst FeatureEncoding is theoretically valid and improves the local score, it results in a major downgrade of the Kaggle score

# Polynomial Features

sklearn.preprocessing.PolynomialFeatures() massively expands the number of features, multiplying each column by every other column, including itself.

We have thus gone from 37 features to 1039 features (almost the same size as dataset rows)

In [5]:
model = FeatureEncoding(features=[ "X_feature_exclude", "X_feature_year_ages", "X_feature_label_encode", "X_feature_onehot", "X_feature_polynomial" ])
(  
    LinearRegressionModel().data['X_train'].shape,
    model.data['X_train'].shape,
)

((1095, 37), (1095, 258))

This also has a major detremental effect on the score 0.1942 -> 0.6264

In [6]:
results = FeatureEncoding.feature_scores([ "X_feature_exclude", "X_feature_year_ages", "X_feature_label_encode", "X_feature_onehot", "X_feature_polynomial" ], depth=1)
for result in reversed(results): print( result )

(0.1978, 'FeatureEncoding', 'LinearRegression', 'X_feature_exclude')
(0.1946, 'FeatureEncoding', 'LinearRegression', 'X_feature_year_ages')
(0.1942, 'FeatureEncoding', 'LinearRegression', 'X_feature_polynomial')
(0.1942, 'FeatureEncoding', 'LinearRegression', '')
(0.1855, 'FeatureEncoding', 'LinearRegression', 'X_feature_label_encode')
(0.1829, 'FeatureEncoding', 'LinearRegression', 'X_feature_onehot')
(0.1823, 'FeatureEncoding', 'LinearRegression', 'X_feature_exclude X_feature_year_ages X_feature_label_encode X_feature_onehot X_feature_polynomial')


In [7]:
Series(model.predict()['LinearRegression']).describe()

count      1459.000000
mean     178764.363497
std       80455.808033
min     -121624.300353
25%      125823.607680
50%      160496.149479
75%      217313.526032
max      948643.487233
dtype: float64

### Hypothesis 
We now have far too many features and are overfitting the data with lots of uncorreleated features. Our polynomial model is also predicting negative prices for some houses.

Now that we have completly trashed our score, it can only get better from here

TODO: Investigate - Regularization: Ridge, Lasso and Elastic Net 

### Submit to Kaggle
```
$ kaggle competitions submit -c house-prices-advanced-regression-techniques -f data/submissions/PolynomialFeatureEncoding.csv -m "FeatureEncoding.py - 1039 Polynomial Features"
```
    
- Your submission scored 2.87097, which is not an improvement of your best score (0.20892). Keep trying!
- Kaggle Rank 3861 / 4486


